In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Suppress tensorflow warnings
from google.colab import drive, files
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image
from PIL import Image

# Set random seed using student ID as required in the assignment
student_id = 404488
np.random.seed(student_id)
tf.random.set_seed(student_id)

In [ ]:
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

In [ ]:
# Mount Google Drive if running in Colab
drive.flush_and_unmount
drive.mount('/content/drive')
!unzip -q '/content/drive/MyDrive/rock-paper-scissors.zip' -d '/content/drive/MyDrive/gesture_dataset'

In [ ]:
# Load images from properly formatted directories
data_dir = '/content/drive/MyDrive/gesture_dataset'
train_dir = os.path.join(data_dir, 'Rock-Paper-Scissors', 'train')
test_dir = os.path.join(data_dir, 'Rock-Paper-Scissors', 'test')

In [ ]:
IMG_SIZE = (224, 224)
batch_size = 32

# Load the full training and test datasets
train_ds_full = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=IMG_SIZE,
    batch_size=None,
    shuffle=False,
    seed=student_id
)

test_ds_full = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=IMG_SIZE,
    batch_size=None,
    shuffle=False,
    seed=student_id
)

# Get class names
class_names = train_ds_full.class_names
print(f"Classes: {class_names}")

In [ ]:
# Combine into one dataset and shuffle
all_images = train_ds_full.concatenate(test_ds_full)
all_images = all_images.shuffle(buffer_size=1000, seed=student_id)

# Get total dataset size
total_size = len(list(all_images))
print(f"Total dataset size: {total_size}")

In [ ]:
train_size = int(0.7 * total_size)
val_size = int(0.1 * total_size)
test_size = total_size - train_size - val_size

# Create the splits
train_ds = all_images.take(train_size).batch(batch_size)
remaining = all_images.skip(train_size)
val_ds = remaining.take(val_size).batch(batch_size)
test_ds = remaining.skip(val_size).batch(batch_size)

# Print dataset sizes
print("Data set sizes:")
print(f"Training set: {tf.data.experimental.cardinality(train_ds)}")
print(f"Validation set: {tf.data.experimental.cardinality(val_ds)}")
print(f"Test set: {tf.data.experimental.cardinality(test_ds)}")

In [ ]:
# Function to count examples per class
def count_examples_per_class(dataset):
    class_counts = [0] * len(class_names)
    for _, labels in dataset:
        for label in labels:
            class_counts[label] += 1
    return class_counts

# Calculate class distribution in each split
train_counts = count_examples_per_class(train_ds)
val_counts = count_examples_per_class(val_ds)
test_counts = count_examples_per_class(test_ds)

In [ ]:
# Plot class distribution
plt.figure(figsize=(12, 6))
plt.suptitle("Class Distribution Across Datasets")
x = np.arange(len(class_names))
width = 0.25

plt.bar(x - width, train_counts, width, label='Training')
plt.bar(x, val_counts, width, label='Validation')
plt.bar(x + width, test_counts, width, label='Test')

plt.xlabel('Class', fontsize=14)
plt.ylabel('Number of Images', fontsize=14)
plt.xticks(x, class_names, fontsize=12)
plt.legend(fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

In [ ]:
# Visualize sample images
plt.figure(figsize=(12, 12))
plt.suptitle("Sample Images from Rock-Paper-Scissors Dataset")

for images, labels in train_ds.take(1):
    for i in range(min(9, len(images))):
        plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()


In [ ]:
# Visualize RGB channel distribution
plt.figure(figsize=(15, 5))
plt.suptitle("RGB Channel Analysis of Sample Images")

for images, labels in train_ds.take(1):
    for i in range(3):
        img = images[i].numpy().astype("uint8")
        class_label = class_names[labels[i]]

        plt.subplot(1, 3, i+1)

        # Plot RGB histograms
        for j, color in enumerate(['red', 'green', 'blue']):
            histogram = plt.hist(img[:,:,j].flatten(), bins=256, alpha=0.5, color=color, label=color)

        plt.title(f"{class_label}")
        plt.xlabel("Pixel Value")
        plt.ylabel("Frequency")
        if i == 0:
            plt.legend()
        plt.ylim(0, 1500)

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

In [ ]:
# Create the directory for test images
!mkdir -p /content/my_test_images

# Upload images
from google.colab import files
uploaded = files.upload()

# Save each file
for fn in uploaded.keys():
   os.rename(fn, f'/content/my_test_images/{fn}')
   print(f"Saved {fn} to /content/my_test_images/{fn}")


In [ ]:
# Create callback for early stopping
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
## Baseline model
model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3)))
model.add(layers.Rescaling(1./255))
model.add(layers.Conv2D(16, 3, padding='same', activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Conv2D(32, 3, padding='same', activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(len(class_names), activation='softmax'))

model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

model.summary()

In [ ]:
# Train the baseline model
start_time = time.time()
baseline_history = model.fit(train_ds, validation_data=val_ds, epochs=100, callbacks=[callback])
baseline_training_time = time.time() - start_time
print(f"\nBaseline model training time: {baseline_training_time:.2f} seconds")

# Evaluate on test set
baseline_test_loss, baseline_test_acc = model.evaluate(test_ds)
print(f"Baseline model test accuracy: {baseline_test_acc:.4f}")

In [ ]:
# Check uploaded files
image_files = [f for f in os.listdir('/content/my_test_images')
              if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

for img_file in image_files:
    # Load and preprocess image
    img_path = os.path.join('/content/my_test_images', img_file)
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img)
    img_array = img_array / 255.0  # Normalize to [0,1]
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

    # Make prediction
    predictions = model.predict(img_array, verbose=0)
    predicted_class_idx = np.argmax(predictions[0])
    predicted_class = class_names[predicted_class_idx]
    confidence = predictions[0][predicted_class_idx] * 100

    # Display image and prediction
    plt.figure(figsize=(8, 6))
    plt.subplot(1, 2, 1)
    plt.imshow(img)
    plt.title(f"File: {img_file}")
    plt.axis('off')

    # Display prediction details
    plt.subplot(1, 2, 2)
    # Create bar chart of predictions
    bars = plt.bar(class_names, predictions[0])
    bars[predicted_class_idx].set_color('red')
    plt.ylim([0, 1.0])
    plt.title(f"Prediction: {predicted_class} ({confidence:.1f}%)")
    plt.xticks(rotation=45)

    plt.tight_layout()
    plt.show()

In [ ]:
# Define data augmentation layers
data_augmentation_layers = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
])

model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3)))
model.add(data_augmentation_layers)
model.add(layers.Rescaling(1./255))
model.add(layers.Conv2D(16, 3, padding='same', activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Conv2D(32, 3, padding='same', activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Conv2D(64, 3, padding='same', activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(len(class_names), activation='softmax'))

model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

model.summary()

In [ ]:
# Train the deeper model
start_time = time.time()
deeper_history = model.fit(train_ds, validation_data=val_ds, epochs=100, callbacks=[callback])
deeper_training_time = time.time() - start_time
print(f"\nDeeper model training time: {deeper_training_time:.2f} seconds")

# Evaluate on test set
deeper_test_loss, deeper_test_acc = model.evaluate(test_ds)
print(f"Deeper model test accuracy: {deeper_test_acc:.4f}")

In [ ]:
# Check uploaded files
image_files = [f for f in os.listdir('/content/my_test_images')
              if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

for img_file in image_files:
    # Load and preprocess image
    img_path = os.path.join('/content/my_test_images', img_file)
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img)
    img_array = img_array / 255.0  # Normalize to [0,1]
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

    # Make prediction
    predictions = model.predict(img_array, verbose=0)
    predicted_class_idx = np.argmax(predictions[0])
    predicted_class = class_names[predicted_class_idx]
    confidence = predictions[0][predicted_class_idx] * 100

    # Display image and prediction
    plt.figure(figsize=(8, 6))
    plt.subplot(1, 2, 1)
    plt.imshow(img)
    plt.title(f"File: {img_file}")
    plt.axis('off')

    # Display prediction details
    plt.subplot(1, 2, 2)
    # Create bar chart of predictions
    bars = plt.bar(class_names, predictions[0])
    bars[predicted_class_idx].set_color('red')
    plt.ylim([0, 1.0])
    plt.title(f"Prediction: {predicted_class} ({confidence:.1f}%)")
    plt.xticks(rotation=45)

    plt.tight_layout()
    plt.show()

In [ ]:
# Wider model with data augmentation and dropout
model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3)))
model.add(data_augmentation_layers)
model.add(layers.Rescaling(1./255))
model.add(layers.Conv2D(32, 3, padding='same', activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Dropout(0.2))
model.add(layers.Conv2D(64, 3, padding='same', activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Dropout(0.2))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(len(class_names), activation='softmax'))

model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

model.summary()

In [ ]:
# Train the wider model
start_time = time.time()
wider_history = model.fit(train_ds, validation_data=val_ds, epochs=100, callbacks=[callback])
wider_training_time = time.time() - start_time
print(f"\nWider model training time: {wider_training_time:.2f} seconds")

# Evaluate on test set
wider_test_loss, wider_test_acc = model.evaluate(test_ds)
print(f"Wider model test accuracy: {wider_test_acc:.4f}")

In [ ]:
# Check uploaded files
image_files = [f for f in os.listdir('/content/my_test_images')
              if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

for img_file in image_files:
    # Load and preprocess image
    img_path = os.path.join('/content/my_test_images', img_file)
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img)
    img_array = img_array / 255.0  # Normalize to [0,1]
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

    # Make prediction
    predictions = model.predict(img_array, verbose=0)
    predicted_class_idx = np.argmax(predictions[0])
    predicted_class = class_names[predicted_class_idx]
    confidence = predictions[0][predicted_class_idx] * 100

    # Display image and prediction
    plt.figure(figsize=(8, 6))
    plt.subplot(1, 2, 1)
    plt.imshow(img)
    plt.title(f"File: {img_file}")
    plt.axis('off')

    # Display prediction details
    plt.subplot(1, 2, 2)
    # Create bar chart of predictions
    bars = plt.bar(class_names, predictions[0])
    bars[predicted_class_idx].set_color('red')
    plt.ylim([0, 1.0])
    plt.title(f"Prediction: {predicted_class} ({confidence:.1f}%)")
    plt.xticks(rotation=45)

    plt.tight_layout()
    plt.show()

In [ ]:
# Compare the three model architectures
architecture_comparison = {
    'Model': ['Baseline CNN', 'Deeper CNN', 'Wider CNN'],
    'Validation Accuracy': [baseline_history.history['val_accuracy'][-1],
                           deeper_history.history['val_accuracy'][-1],
                           wider_history.history['val_accuracy'][-1]],
    'Test Accuracy': [baseline_test_acc, deeper_test_acc, wider_test_acc],
    'Training Time (s)': [baseline_training_time, deeper_training_time, wider_training_time]
}

In [ ]:
# Create a bar chart to compare accuracies
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.bar(architecture_comparison['Model'], architecture_comparison['Test Accuracy'])
plt.title('Test Accuracy by Architecture')
plt.xlabel('Model Architecture')
plt.ylabel('Test Accuracy')
plt.ylim(0, 1)

# Create a bar chart to compare training times
plt.subplot(1, 2, 2)
plt.bar(architecture_comparison['Model'], architecture_comparison['Training Time (s)'])
plt.title('Training Time by Architecture')
plt.xlabel('Model Architecture')
plt.ylabel('Training Time (seconds)')

plt.tight_layout()
plt.show()

In [ ]:
# Print the comparison as a formatted table
print("\nModel Architecture Comparison:")
for i in range(len(architecture_comparison['Model'])):
    model_name = architecture_comparison['Model'][i]
    val_acc = architecture_comparison['Validation Accuracy'][i]
    test_acc = architecture_comparison['Test Accuracy'][i]
    time = architecture_comparison['Training Time (s)'][i]
    print(f"{model_name}: Val Accuracy = {val_acc:.4f}, Test Accuracy = {test_acc:.4f}, Training Time = {time:.2f} seconds")

In [ ]:
def create_wider_model():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3)))
    model.add(data_augmentation_layers)
    model.add(layers.Rescaling(1./255))
    model.add(layers.Conv2D(32, 3, padding='same', activation='relu'))
    model.add(layers.MaxPooling2D())
    model.add(layers.Dropout(0.2))
    model.add(layers.Conv2D(64, 3, padding='same', activation='relu'))
    model.add(layers.MaxPooling2D())
    model.add(layers.Dropout(0.2))
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(len(class_names), activation='softmax'))
    return model

In [ ]:
# Learning rates to test
learning_rates = [0.01, 0.001, 0.0001]
learning_rate_histories = []
learning_rate_accuracies = []
learning_rate_times = []

print("\n===== Testing Different Learning Rates on Wider Model =====")

for lr in learning_rates:
    print(f"\nTraining with learning rate: {lr}")

    # Create new model instance
    model = create_wider_model()

    # Compile with current learning rate
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                  metrics=['accuracy'])

    # Short summary (only for the first learning rate)
    if lr == learning_rates[0]:
        model.summary()

    # Train the model
    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=30,  # Reduced to save time while testing multiple learning rates
        callbacks=[callback],
        verbose=1
    )

    # Evaluate on test set
    test_loss, test_acc = model.evaluate(test_ds)
    print(f"Test accuracy: {test_acc:.4f}")

    # Store results
    learning_rate_histories.append(history)
    learning_rate_accuracies.append(test_acc)

# Find the best learning rate
best_lr_index = np.argmax(learning_rate_accuracies)
best_lr = learning_rates[best_lr_index]
best_lr_accuracy = learning_rate_accuracies[best_lr_index]

In [ ]:
print(f"\nBest learning rate: {best_lr} with test accuracy: {best_lr_accuracy:.4f}")

# Visualize learning rate performance
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.bar([str(lr) for lr in learning_rates], learning_rate_accuracies)
plt.title('Test Accuracy by Learning Rate')
plt.xlabel('Learning Rate')
plt.ylabel('Test Accuracy')
plt.ylim(0, 1)

plt.subplot(1, 2, 2)
for i, lr in enumerate(learning_rates):
    plt.plot(learning_rate_histories[i].history['val_accuracy'], label=f'LR: {lr}')
plt.title('Validation Accuracy During Training')
plt.xlabel('Epoch')
plt.ylabel('Validation Accuracy')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Now train the wider model with the best learning rate for the full number of epochs
print(f"\nTraining the wider model with the best learning rate: {best_lr}")
model = create_wider_model()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=best_lr),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

# Train the wider model
wider_history = model.fit(train_ds, validation_data=val_ds, epochs=100, callbacks=[callback])

# Evaluate on test set
wider_test_loss, wider_test_acc = model.evaluate(test_ds)
print(f"Wider model test accuracy: {wider_test_acc:.4f}")


In [ ]:
# Check uploaded files
image_files = [f for f in os.listdir('/content/my_test_images')
              if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

for img_file in image_files:
    # Load and preprocess image
    img_path = os.path.join('/content/my_test_images', img_file)
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img)
    img_array = img_array / 255.0  # Normalize to [0,1]
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

    # Make prediction
    predictions = model.predict(img_array, verbose=0)
    predicted_class_idx = np.argmax(predictions[0])
    predicted_class = class_names[predicted_class_idx]
    confidence = predictions[0][predicted_class_idx] * 100

    # Display image and prediction
    plt.figure(figsize=(8, 6))
    plt.subplot(1, 2, 1)
    plt.imshow(img)
    plt.title(f"File: {img_file}")
    plt.axis('off')

    # Display prediction details
    plt.subplot(1, 2, 2)
    # Create bar chart of predictions
    bars = plt.bar(class_names, predictions[0])
    bars[predicted_class_idx].set_color('red')
    plt.ylim([0, 1.0])
    plt.title(f"Prediction: {predicted_class} ({confidence:.1f}%)")
    plt.xticks(rotation=45)

    plt.tight_layout()
    plt.show()